In [155]:
import torch
from sklearn.calibration import CalibratedClassifierCV
import torch.optim as optim
import torch.nn as nn
import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


# Get Data and Base Model

In [156]:
dataset_name = 'CUB'
concept_name = 'has_back_color::black'

metadata_df = pd.read_csv(f'../Data/{dataset_name}/metadata.csv')
embeddings = torch.load(f'Embeddings/{dataset_name}/embeddings.pt') 

train_indices = metadata_df[metadata_df['split'] == 'train'].index.tolist()
test_indices = metadata_df[metadata_df['split'] == 'test'].index.tolist()
calibration_indices = metadata_df[metadata_df['split'] == 'calibration'].index.tolist()

X_train = embeddings[train_indices].numpy()
X_test = embeddings[test_indices].numpy()
X_cal = embeddings[calibration_indices].numpy()

y_train = np.array(metadata_df[(metadata_df['split'] == 'train')][concept_name])
y_test = np.array(metadata_df[(metadata_df['split'] == 'test')][concept_name])
y_cal = np.array(metadata_df[(metadata_df['split'] == 'calibration')][concept_name])

In [7]:
# for now, train a basic classifier

In [157]:
base_model = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=200, random_state=42)
base_model.fit(X_train, y_train)
y_pred = base_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 74.69%


## Platt Scaling

In [163]:
platt_calibrated_model = CalibratedClassifierCV(base_model, method='sigmoid', cv='prefit')
platt_calibrated_model.fit(X_cal, y_cal)  # Fit calibration on validation set

y_probs = platt_calibrated_model.predict_proba(X_test)
y_pred = np.argmax(y_probs, axis=1)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 75.70%


## Isotonic Regression

In [164]:
isotonic_calibrated_model = CalibratedClassifierCV(base_model, method='isotonic', cv='prefit') 
isotonic_calibrated_model.fit(X_cal, y_cal)

y_probs = isotonic_calibrated_model.predict_proba(X_test)
y_pred = np.argmax(y_probs, axis=1)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 75.65%


## Temperature Scaling

In [197]:
class TemperatureScaling(nn.Module):
    """A simple module for temperature scaling."""
    def __init__(self, base_model):
        super(TemperatureScaling, self).__init__()
        self.base_model = base_model
        
    def predict_scaled_logits(self, X):
        """Scale the logits using the current temperature."""
        original_logits = self.base_model.predict_proba(X) 
        print(original_logits[:5])
        scaled_logits = original_logits / self.temperature  # Scale by the temperature
        return scaled_logits
        
    def predict_proba(self, X):
        """Return the probability predictions from the scaled logits."""
        scaled_logits = self.predict_scaled_logits(X)  # Get scaled logits
        exps = np.exp(scaled_logits - np.max(scaled_logits, axis=1, keepdims=True))  # Stability trick for softmax
        return exps / np.sum(exps, axis=1, keepdims=True)  # Normalize to get probabilities

def train_temperature_scaling(base_model, X_cal, y_cal):
    """Train temperature scaling using negative log-likelihood. """
    
    temperature_model = TemperatureScaling(base_model)  # Initialize the temperature scaling model
    
    def nll_loss(temperature):
        """Negative log-likelihood loss for the given temperature."""
        temperature_model.temperature = temperature[0]  # Update temperature
        scaled_logits = temperature_model.predict_scaled_logits(X_cal)  # Get scaled logits
        #print(f"Scaled logits for temperature {temperature[0]}: {scaled_logits[:5]}") 
        # Compute cross-entropy loss
        log_probs = scaled_logits - np.log(np.sum(np.exp(scaled_logits), axis=1, keepdims=True))
        nll = -np.mean(log_probs[np.arange(len(y_cal)), y_cal])  # Negative log-likelihood
        #print("Current loss:", nll)
        return nll
    
    # Minimize the negative log-likelihood loss to find the optimal temperature
    result = minimize(nll_loss, x0=[10], bounds=[(1e-2, 10.0)], tol=1, method='L-BFGS-B')  # Temperature > 0
    
    # To visualize the loss for different temperatures
    # temperatures = np.linspace(0.1, 10, 100)
    # losses = [nll_loss([t]) for t in temperatures]
    # plt.plot(temperatures, losses)
    # plt.xlabel('Temperature')
    # plt.ylabel('Negative Log-Likelihood')
    # plt.show()
    
    # Store the optimal temperature
    temperature_model.temperature = result.x[0]  
    print(f"Optimal temperature: {temperature_model.temperature:.4f}")
    
    return temperature_model

temperature_calibrated_model = train_temperature_scaling(base_model, X_cal, y_cal)

y_probs = temperature_calibrated_model.predict_proba(X_test)
y_pred = np.argmax(y_probs, axis=1)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

[[1.0000000e+00 4.3254425e-10]
 [9.9957806e-01 4.2194437e-04]
 [9.8677015e-01 1.3229854e-02]
 [9.9973083e-01 2.6914920e-04]
 [1.0000000e+00 3.4185834e-09]]
[[1.0000000e+00 4.3254425e-10]
 [9.9957806e-01 4.2194437e-04]
 [9.8677015e-01 1.3229854e-02]
 [9.9973083e-01 2.6914920e-04]
 [1.0000000e+00 3.4185834e-09]]
Optimal temperature: 10.0000
[[1.0000000e+00 1.9171861e-10]
 [1.0000000e+00 1.7745098e-08]
 [9.9999529e-01 4.6975442e-06]
 [9.9957001e-01 4.3001771e-04]
 [9.9999952e-01 4.9182859e-07]]
Test Accuracy: 74.69%
